### Cos'è `prepare_data`?

La funzione `prepare_data` è responsabile di eseguire tutti i passaggi preparatori prima di iniziare l'algoritmo genetico vero e proprio. Analizziamola passo per passo:

#### 1. Caricamento dei dati

```python
data = np.load(file_path)
X = data['x']  # Input (valori delle variabili)
y = data['y']  # Output (risultato che vogliamo predire)
```

Questo carica i dati dal file .npz. `X` contiene i valori delle variabili indipendenti (input) e `y` contiene i valori della variabile dipendente (output).

#### 2. Analisi della dimensionalità

```python
if X.ndim == 1:
    n_features = 1
else:
    n_features = X.shape[1]
```

Qui determiniamo se stiamo lavorando con una singola variabile (ad es. `y = f(x)`) o con più variabili (ad es. `y = f(x₁, x₂, x₃, ...)`). Questo è fondamentale perché:
- Con una variabile, le nostre espressioni saranno del tipo `2*x + 3`
- Con più variabili, saranno tipo `2*x[0] + 3*x[1] - 5`

#### 3. Calcolo delle statistiche

```python
print(f"- Media: {np.mean(y):.4f}")
print(f"- Min: {np.min(y):.4f}, Max: {np.max(y):.4f}")
```

Queste statistiche ci aiutano a capire la natura dei dati: la scala, il range, ecc.


#### 4. Configurazione per il GP

```python
variables = [f'x[{i}]' for i in range(n_features)]
const_range = max(np.max(np.abs(X)), np.max(np.abs(y)))
```

Qui creiamo i nomi delle variabili (`x[0]`, `x[1]`, ecc.) e definiamo il range per le costanti.

### Cosa sono le costanti e perché sono importanti?

#### Le costanti nell'ambito della Symbolic Regression

Nel contesto della programmazione genetica per symbolic regression, le espressioni matematiche contengono:

1. **Variabili** (come `x` o `x[0]`, `x[1]`): Questi sono i valori di input che cambiano per ogni punto dati.

2. **Operatori** (come `+`, `-`, `*`, `/`, `sin`, `exp`): Sono le operazioni matematiche.

3. **Costanti** (come `2.5`, `3.14`, `-0.7`): Sono numeri fissi che completano l'espressione.

**Esempio**: In un'espressione come `2.5 * x² + 3.14 * sin(x) - 0.7`, i numeri `2.5`, `3.14` e `-0.7` sono costanti.

#### A cosa serve `const_range`?

`const_range` determina il range entro cui verranno generate casualmente le costanti durante:

1. **Inizializzazione della popolazione**: Quando creiamo le espressioni iniziali, dobbiamo inserire costanti ragionevoli.
   - Se i dati sono nell'ordine di 0.001, costanti come 1000 sarebbero inutili
   - Se i dati sono nell'ordine di 1000, costanti come 0.001 sarebbero inefficaci

2. **Mutazione delle costanti**: Durante l'evoluzione, alcune mutazioni modificheranno i valori delle costanti.

```python
# Esempio di generazione di una costante casuale
random_constant = random.uniform(-const_range, const_range)
# o con distribuzione più centrata
random_constant = random.uniform(-const_range/2, const_range/2)
```

#### Perché calcoliamo `const_range` dai dati?

Calcoliamo `const_range` in base ai valori massimi assoluti di X e y per:

1. **Efficienza**: Costanti troppo grandi o troppo piccole rispetto ai dati rendono l'evoluzione inefficiente.

2. **Convergenza più rapida**: Con costanti nel range giusto, l'algoritmo trova più velocemente soluzioni valide.

3. **Evitare problemi numerici**: Costanti troppo estreme possono causare overflow/underflow con operazioni come potenze o esponenziali.

### Esempio pratico

Supponiamo di voler trovare una funzione che rappresenti la relazione `y = 2x² + 3.5`:

- I valori di X potrebbero essere tra -10 e 10
- I valori di y sarebbero quindi tra 0 e 200+
- `const_range` sarebbe circa 200
- La GP genererà costanti casuali tipo 53.2, -47.8, 122.6, ecc.
- Con l'evoluzione, alcune si avvicineranno ai valori reali (2 e 3.5)

In breve, le costanti sono i "numeri" nelle espressioni matematiche, e `const_range` aiuta l'algoritmo a generare questi numeri in modo sensato rispetto ai dati che sta analizzando.

Ti è più chiaro ora? Hai altre domande specifiche sulla funzione `prepare_data` o sul ruolo delle costanti?

In [1]:
import numpy as np
import random
from typing import List, Union, Callable, Optional, Tuple, Dict, Any
import time
import copy
import matplotlib.pyplot as plt
from icecream import ic
from tqdm.auto import tqdm

c:\Users\domen\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Data Preparation

In [2]:
def prepare_data(file_path):
    """
    Prepara i dati per la Symbolic Regression con Genetic Programming
    
    Args:
        file_path: Percorso del file .npz con i dati
        
    Returns:
        X: Array di input features
        y: Array di output target
        config: Configurazione per il GP basata sui dati
    """
    # 1. Caricamento dei dati
    print(f"Caricamento dati da {file_path}...")
    data = np.load(file_path)
    X = data['x']
    y = data['y']
    
    # Stampa forme originali per debug
    print(f"Forme originali: X shape {X.shape}, y shape {y.shape}")
    
    # Gestione specifica per dati con dimensioni trasposte
    # Se abbiamo più features che campioni, probabilmente dobbiamo trasporre
    if X.ndim > 1 and X.shape[1] > X.shape[0] and y.shape[0] == X.shape[1]:
        print("Rilevato formato con features sulle righe e campioni sulle colonne, trasposizione...")
        X = X.T  # Trasponiamo per avere i campioni sulle righe
        print(f"X trasposto ha forma {X.shape}")
    
    # Gestione di formati diversi
    if y.ndim > 1:
        if y.shape[0] == 1 or y.shape[1] == 1:  # Se y è [1, n_samples] o [n_samples, 1]
            y = y.flatten()
            print(f"y trasformato in forma {y.shape}")
    
    # Assicuriamo che X sia 2D se multidimensionale
    if X.ndim == 1:
        X = X.reshape(-1, 1)
        print(f"X reso 2D con forma {X.shape}")
    
    print(f"Forme finali: X shape {X.shape}, y shape {y.shape}")
    
    # Verifica di coerenza: il numero di campioni deve corrispondere
    if X.shape[0] != len(y):
        raise ValueError(f"Numero di campioni non coerente: X ha {X.shape[0]} campioni, y ne ha {len(y)}")
    
    # 2. Analisi dei dati
    # Determinare la dimensionalità dell'input
    n_features = X.shape[1]
    print(f"Input {n_features}-dimensionale con {X.shape[0]} campioni")
    
    # 3. Configurazione per GP
    # Definiamo il set di variabili in base alla dimensionalità
    variables = [f'x[{i}]' for i in range(n_features)]
    
    # Determiniamo un range ragionevole per le costanti casuali
    const_range = max(np.max(np.abs(X)), np.max(np.abs(y)))
    
    # Configurazione completa per il GP
    config = {
        'variables': variables,
        'n_features': n_features,
        'const_range': const_range,
        'y_stats': {
            'mean': float(np.mean(y)),
            'std': float(np.std(y)),
            'min': float(np.min(y)),
            'max': float(np.max(y))
        },
        'dataset_size': len(y)
    }
    
    return X, y, config

for i in range(0, 9):
    file_path = f"../data/problem_{i}.npz"
    X, y, config = prepare_data(file_path)
    print(f"\nConfigurazione per GP: {config}")
    print("-" * 50)

Caricamento dati da ../data/problem_0.npz...
Forme originali: X shape (2, 1000), y shape (1000,)
Rilevato formato con features sulle righe e campioni sulle colonne, trasposizione...
X trasposto ha forma (1000, 2)
Forme finali: X shape (1000, 2), y shape (1000,)
Input 2-dimensionale con 1000 campioni

Configurazione per GP: {'variables': ['x[0]', 'x[1]'], 'n_features': 2, 'const_range': np.float64(3.23346517158693), 'y_stats': {'mean': 0.0740349656325135, 'std': 1.8421127520800509, 'min': -3.208666606823163, 'max': 3.23346517158693}, 'dataset_size': 1000}
--------------------------------------------------
Caricamento dati da ../data/problem_1.npz...
Forme originali: X shape (1, 500), y shape (500,)
Rilevato formato con features sulle righe e campioni sulle colonne, trasposizione...
X trasposto ha forma (500, 1)
Forme finali: X shape (500, 1), y shape (500,)
Input 1-dimensionale con 500 campioni

Configurazione per GP: {'variables': ['x[0]'], 'n_features': 1, 'const_range': np.float64(0.

### Expression Representation

In [3]:
class Node:
    """Classe base per rappresentare un nodo nell'albero di espressioni"""
    def __init__(self):
        self.depth = 0  # Profondità del nodo
    
    def evaluate(self, X: np.ndarray) -> np.ndarray:
        """Valuta il nodo dato un input X"""
        raise NotImplementedError("Devi implementare il metodo evaluate nella sottoclasse")
    def copy(self) -> 'Node':
        """Copia il nodo corrente"""
        raise NotImplementedError("Devi implementare il metodo copy nella sottoclasse")
    
    def to_string(self) -> str:
        """Restituisce una rappresentazione stringa del nodo"""
        raise NotImplementedError("Devi implementare il metodo __str__ nella sottoclasse")
    
    def get_complexity(self) -> int:
        """Restituisce la complessità del nodo (numero di nodi)"""
        return NotImplementedError("Devi implementare il metodo get_complexity nella sottoclasse")
    
    def get_height(self) -> int:
        """Restituisce l'altezza del nodo"""
        return NotImplementedError("Devi implementare il metodo get_height nella sottoclasse")
    
    def get_nodes(self) -> int:
        """Restituisce il numero totale di nodi nell'albero"""
        return NotImplementedError("Devi implementare il metodo get_nodes nella sottoclasse")
    

In [4]:
class FunctionNode(Node):
    """Classe per rappresentare un nodo funzione nell'albero di espressioni"""
    def __init__(self, function: Callable, arity: int, symbol: str, children: List[Node] = None):
        super().__init__()
        self.function = function # Funzione (np) da applicare
        self.arity = arity # Numero di argomenti richiesti dalla funzione
        self.symbol = symbol # Simbolo per la rappresentazione stringa
        self.children = children if children is not None else []

    def evaluate(self, X: np.ndarray)->np.ndarray:
        """Valuta la funzione applicandola ai risultati dei figli"""
        # Valuta i figli
        args= [child.evaluate(X) for child in self.children]
        # Applica la funzione
        return self.function(*args)
    
    def copy(self) -> 'FunctionNode':
        """Crea una copia profonda del nodo funzione"""
        new_children = [child.copy() for child in self.children]
        new_node = FunctionNode(self.function, self.arity, self.symbol, new_children)
        new_node.depth = self.depth
        return new_node
    
    def to_string(self) -> str:
        """Restituisce una rappresentazione stringa del nodo funzione"""
        if self.arity == 1:
            # Funzione unaria
            return f"{self.symbol}({self.children[0].to_string()})"
        elif self.arity == 2:
            # Funzione binaria (es. +, -, *, /)
            return f"({self.children[0].to_string()} {self.symbol} {self.children[1].to_string()})"
        else:
            # Funzioni con arità maggiore (anche se non dovrebbero essere comuni)
            args = ", ".join(child.to_string() for child in self.children)
            return f"{self.symbol}({args})"
        
    def get_complexity(self) -> int:
        """Restituisce la complessità del nodo (numero di nodi)"""
        return 1 + sum(child.get_complexity() for child in self.children)

    def get_height(self) -> int:
        """Restituisce l'altezza del nodo"""
        return 1 + max((child.get_height() for child in self.children), default=0)

    def get_nodes(self) -> List[Node]:
        """Restituisce una lista di tutti i nodi nell'albero"""
        nodes = [self]
        for child in self.children:
            nodes.extend(child.get_nodes())
        return nodes
    


In [5]:
class TerminalNode(Node):
    """Rappresenta un nodo terminale nell'albero (variabile o costante)"""
    
    def __init__(self, value, is_variable: bool = False, var_index: int = None):
        super().__init__()
        self.value = value
        self.is_variable = is_variable
        self.var_index = var_index  # usato solo se is_variable è True
    
    def evaluate(self, X: np.ndarray) -> np.ndarray:
        """Valuta il nodo terminale"""
        if self.is_variable:
            # Se è una variabile, prendiamo il valore dall'input X
            if X.ndim == 1 and self.var_index == 0:
                return X  # caso speciale per input 1D
            else:
                return X[:, self.var_index]
        else:
            # Se è una costante, restituiamo il valore (broadcast su tutti i campioni)
            return np.full(X.shape[0] if X.ndim > 1 else len(X), self.value)
    
    def copy(self) -> 'TerminalNode':
        """Crea una copia del nodo terminale"""
        new_node = TerminalNode(self.value, self.is_variable, self.var_index)
        new_node.depth = self.depth
        return new_node
    
    def to_string(self) -> str:
        """Restituisce la rappresentazione in forma di stringa del nodo"""
        if self.is_variable:
            return f"x[{self.var_index}]"
        else:
            # Formattazione delle costanti per evitare numeri troppo lunghi
            if isinstance(self.value, float):
                return f"{self.value:.4f}"
            return str(self.value)
    
    def get_complexity(self) -> int:
        """La complessità di un nodo terminale è 1"""
        return 1
    
    def get_height(self) -> int:
        """L'altezza di un nodo terminale è 0"""
        return 0
    
    def get_nodes(self) -> List[Node]:
        """Restituisce una lista contenente solo questo nodo"""
        return [self]


In [6]:
class ExpressionTree:
    """Rappresenta un albero di espressione completo"""
    
    def __init__(self, root: Node):
        self.root = root
        self.update_node_depths()
        self.fitness = None
        self.adjusted_fitness = None  # per fitness sharing e altre tecniche
        self.age = 0  # per age layering
    
    def evaluate(self, X: np.ndarray) -> np.ndarray:
        """Valuta l'albero di espressione sui dati di input"""
        return self.root.evaluate(X)
    
    def copy(self) -> 'ExpressionTree':
        """Crea una copia profonda dell'albero"""
        new_tree = ExpressionTree(self.root.copy())
        new_tree.fitness = self.fitness
        new_tree.adjusted_fitness = self.adjusted_fitness
        new_tree.age = self.age
        return new_tree
    
    def to_string(self) -> str:
        """Restituisce la rappresentazione in forma di stringa dell'albero"""
        return self.root.to_string()
    
    def get_complexity(self) -> int:
        """Restituisce la complessità dell'albero"""
        return self.root.get_complexity()
    
    def get_height(self) -> int:
        """Restituisce l'altezza dell'albero"""
        return self.root.get_height()
    
    def get_nodes(self) -> List[Node]:
        """Restituisce una lista di tutti i nodi nell'albero"""
        return self.root.get_nodes()
    
    def get_subtree_at_index(self, index: int) -> Node:
        """
        Restituisce il sottoalbero al nodo specificato dall'indice
        Utile per le operazioni di crossover e mutazione
        """
        nodes = self.get_nodes()
        if 0 <= index < len(nodes):
            return nodes[index]
        return None
    
    def replace_subtree_at_index(self, index: int, new_subtree: Node) -> bool:
        """
        Sostituisce il sottoalbero al nodo specificato dall'indice
        Restituisce True se l'operazione è riuscita, False altrimenti
        """
        nodes = self.get_nodes()
        if not (0 <= index < len(nodes)):
            return False
        
        target_node = nodes[index]
        
        # Caso speciale: sostituire la radice dell'albero
        if target_node == self.root:
            self.root = new_subtree
            self.update_node_depths()
            return True
        
        # Altrimenti, dobbiamo trovare il genitore del nodo target
        for node in nodes:
            if isinstance(node, FunctionNode):
                for i, child in enumerate(node.children):
                    if child == target_node:
                        node.children[i] = new_subtree
                        self.update_node_depths()
                        return True
        
        return False
    
    def update_node_depths(self):
        """Aggiorna la profondità di tutti i nodi nell'albero"""
        self._update_depth(self.root, 0)
    
    def _update_depth(self, node: Node, depth: int):
        """Helper ricorsivo per aggiornare la profondità"""
        node.depth = depth
        if isinstance(node, FunctionNode):
            for child in node.children:
                self._update_depth(child, depth + 1)

### Function Set

In [7]:
def safe_div(a: np.ndarray, b: np.ndarray) -> np.ndarray:
    """Divisione protetta: restituisce a/b o 1 quando b è vicino a zero"""
    return np.divide(a, b, out=np.ones_like(a), where=np.abs(b) > 1e-10)

def safe_log(a: np.ndarray) -> np.ndarray:
    """Logaritmo protetto: restituisce log(|a|) o 0 per a vicino a zero"""
    return np.log(np.abs(a), out=np.zeros_like(a), where=np.abs(a) > 1e-10)

def safe_sqrt(a: np.ndarray) -> np.ndarray:
    """Radice quadrata protetta: restituisce sqrt(|a|)"""
    return np.sqrt(np.abs(a))

def safe_exp(a: np.ndarray) -> np.ndarray:
    """Esponenziale protetto: limita l'input per evitare overflow"""
    # Limita gli input a [-200, 100] per evitare overflow
    return np.exp(np.clip(a, -200, 200))

def safe_sin(a: np.ndarray) -> np.ndarray:
    """Seno protetto"""
    return np.sin(np.clip(a, -1000, 1000))

def safe_cos(a: np.ndarray) -> np.ndarray:
    """Coseno protetto"""
    return np.cos(np.clip(a, -1000, 1000))

def safe_tan(a: np.ndarray) -> np.ndarray:
    """Tangente protetta: limita gli output per evitare valori estremi"""
    return np.clip(np.tan(a), -200, 200)


In [8]:
def create_function_set(use_trig: bool = True, use_exp_log: bool = True) -> List[Dict[str, Any]]:
    """
    Crea un set di funzioni da utilizzare nell'albero delle espressioni
    
    Args:
        use_trig: Se includere funzioni trigonometriche
        use_exp_log: Se includere funzioni esponenziali e logaritmiche
        
    Returns:
        Lista di dizionari, ciascuno con:
            - function: la funzione Python da chiamare
            - arity: il numero di argomenti richiesti
            - symbol: il simbolo per la visualizzazione
            - weight: peso di selezione (probabilità relativa)
    """
    # Funzioni aritmetiche di base (sempre incluse)
    functions = [
        {'function': np.add, 'arity': 2, 'symbol': '+', 'weight': 1.0},
        {'function': np.subtract, 'arity': 2, 'symbol': '-', 'weight': 1.0},
        {'function': np.multiply, 'arity': 2, 'symbol': '*', 'weight': 1.0},
        {'function': safe_div, 'arity': 2, 'symbol': '/', 'weight': 0.7},  # Peso più basso per la divisione
    ]
    
    # Funzioni trigonometriche (opzionali)
    if use_trig:
        functions.extend([
            {'function': safe_sin, 'arity': 1, 'symbol': 'sin', 'weight': 0.6},
            {'function': safe_cos, 'arity': 1, 'symbol': 'cos', 'weight': 0.6},
            {'function': safe_tan, 'arity': 1, 'symbol': 'tan', 'weight': 0.6},  # Peso più basso per la tangente
        ])
    
    # Funzioni esponenziali e logaritmiche (opzionali)
    if use_exp_log:
        functions.extend([
            {'function': safe_exp, 'arity': 1, 'symbol': 'exp', 'weight': 0.4},
            {'function': safe_log, 'arity': 1, 'symbol': 'log', 'weight': 0.5},
            {'function': safe_sqrt, 'arity': 1, 'symbol': 'sqrt', 'weight': 0.6},
        ])
    
    return functions


### Terminal Set

In [9]:
def create_variable_terminals(n_features: int) -> List[Dict[str, Any]]:
    """
    Crea terminali per le variabili d'input
    
    Args:
        n_features: Numero di variabili d'input
        
    Returns:
        Lista di dizionari per i terminali variabili
    """
    return [
        {
            'is_variable': True, 
            'var_index': i, 
            'weight': 1.0
        } for i in range(n_features)
    ]

In [10]:
def create_constant_terminals(const_range: float, n_constants: int = 10) -> List[Dict[str, Any]]:
    """
    Crea terminali per le costanti
    
    Args:
        const_range: Range per le costanti casuali
        n_constants: Numero di costanti pre-generate
        
    Returns:
        Lista di dizionari per i terminali costanti
    """
    # Costanti fisse importanti
    fixed_constants = [
        {'is_variable': False, 'value': 0.0, 'weight': 0.5},
        {'is_variable': False, 'value': 1.0, 'weight': 0.5},
        {'is_variable': False, 'value': -1.0, 'weight': 0.3},
        {'is_variable': False, 'value': np.pi, 'weight': 0.2},
        {'is_variable': False, 'value': np.e, 'weight': 0.2},
    ]
    
    # Costanti casuali pre-generate
    random_constants = [
        {
            'is_variable': False, 
            'value': random.uniform(-const_range, const_range), 
            'weight': 0.3
        } for _ in range(n_constants)
    ]
    
    return fixed_constants + random_constants


In [11]:
def generate_ephemeral_constant(const_range: float) -> float:
    """
    Genera una costante casuale (ephemeral random constant)
    Può essere chiamata durante l'inizializzazione dell'albero
    
    Args:
        const_range: Range per le costanti casuali
        
    Returns:
        Valore della costante generata
    """
    return random.uniform(-const_range, const_range)

In [12]:
class GPConfig:
    """Classe per gestire la configurazione dell'algoritmo GP"""
    
    def __init__(self, 
                 n_features: int,
                 const_range: float,
                 use_trig: bool = True,
                 use_exp_log: bool = True,
                 min_depth: int = 2,
                 max_depth: int = 6,
                 pop_size: int = 500,
                 generations: int = 50,
                 tournament_size: int = 5,
                 crossover_prob: float = 0.7,
                 mutation_prob: float = 0.2,
                 elitism_rate: float = 0.1,
                 max_tree_size: int = 50,
                 parsimony_coef: float = 0.01,
                 diversity_weight: float = 0.2):
        
        # Configurazione del set di funzioni e terminali
        self.function_set = create_function_set(use_trig, use_exp_log)
        self.variable_terminals = create_variable_terminals(n_features)
        self.constant_terminals = create_constant_terminals(const_range)
        
        # Calcola pesi cumulativi per la selezione pesata
        self._calculate_weights()
        
        # Limiti di dimensione dell'albero
        self.min_depth = min_depth
        self.max_depth = max_depth
        self.max_tree_size = max_tree_size
        
        # Parametri dell'algoritmo evolutivo
        self.pop_size = pop_size
        self.generations = generations
        self.tournament_size = tournament_size
        self.crossover_prob = crossover_prob
        self.mutation_prob = mutation_prob
        self.elitism_rate = elitism_rate
        
        # Controllo del bloat e mantenimento diversità
        self.parsimony_coef = parsimony_coef  # penalità per la complessità
        self.diversity_weight = diversity_weight  # peso per la diversità
        
        # Altri parametri
        self.n_features = n_features
        self.const_range = const_range
        
    def _calculate_weights(self):
        """Calcola i pesi cumulativi per la selezione pesata di funzioni e terminali"""
        # Pesi cumulativi per le funzioni
        cum_weight = 0
        self.function_weights = []
        for func in self.function_set:
            cum_weight += func['weight']
            self.function_weights.append(cum_weight)
        
        # Normalizza i pesi
        if cum_weight > 0:
            self.function_weights = [w / cum_weight for w in self.function_weights]
        
        # Pesi cumulativi per i terminali variabili
        cum_weight = 0
        self.variable_weights = []
        for var in self.variable_terminals:
            cum_weight += var['weight']
            self.variable_weights.append(cum_weight)
        
        # Normalizza i pesi
        if cum_weight > 0:
            self.variable_weights = [w / cum_weight for w in self.variable_weights]
        
        # Pesi cumulativi per i terminali costanti
        cum_weight = 0
        self.constant_weights = []
        for const in self.constant_terminals:
            cum_weight += const['weight']
            self.constant_weights.append(cum_weight)
        
        # Normalizza i pesi
        if cum_weight > 0:
            self.constant_weights = [w / cum_weight for w in self.constant_weights]
    
    def get_random_function(self) -> Dict[str, Any]:
        """Seleziona casualmente una funzione dal set, basandosi sui pesi"""
        r = random.random()
        for i, w in enumerate(self.function_weights):
            if r <= w:
                return self.function_set[i]
        return self.function_set[-1]  # fallback
    
    def get_random_variable(self) -> Dict[str, Any]:
        """Seleziona casualmente una variabile terminale dal set, basandosi sui pesi"""
        if not self.variable_terminals:
            raise ValueError("Non ci sono variabili disponibili")
        
        r = random.random()
        for i, w in enumerate(self.variable_weights):
            if r <= w:
                return self.variable_terminals[i]
        return self.variable_terminals[-1]  # fallback
    
    def get_random_constant(self) -> Dict[str, Any]:
        """Seleziona casualmente una costante terminale dal set, basandosi sui pesi"""
        if not self.constant_terminals:
            # Genera una costante efimera se non ci sono constanti predefinite
            return {'is_variable': False, 'value': generate_ephemeral_constant(self.const_range)}
        
        # Occasionalmente genera una nuova costante efimera anziché usarne una predefinita
        if random.random() < 0.3:  # 30% di probabilità di generare una nuova costante
            return {'is_variable': False, 'value': generate_ephemeral_constant(self.const_range)}
        
        # Altrimenti seleziona dalle costanti predefinite
        r = random.random()
        for i, w in enumerate(self.constant_weights):
            if r <= w:
                return self.constant_terminals[i]
        return self.constant_terminals[-1]  # fallback
    
    def get_random_terminal(self) -> Dict[str, Any]:
        """Seleziona casualmente un terminale (variabile o costante)"""
        # Probabilità di selezionare una variabile vs una costante
        # In genere vogliamo dare più peso alle variabili
        if random.random() < 0.7:  # 70% di probabilità di selezionare una variabile
            try:
                return self.get_random_variable()
            except ValueError:
                return self.get_random_constant()
        else:
            return self.get_random_constant()

### Initial Population

In [13]:
def grow_tree(config: GPConfig, max_depth: int, min_depth: int = 1, current_depth: int = 0) -> Node:
    """
    Metodo 'grow' per generare un albero con profondità variabile
    
    Args:
        config: Configurazione del GP
        max_depth: Profondità massima dell'albero
        min_depth: Profondità minima dell'albero
        current_depth: Profondità corrente del nodo
        
    Returns:
        Nodo radice dell'albero generato
    """
    # Se siamo alla profondità massima, creiamo solo nodi terminali
    if current_depth >= max_depth:
        terminal_info = config.get_random_terminal()
        if terminal_info['is_variable']:
            return TerminalNode(None, is_variable=True, var_index=terminal_info['var_index'])
        else:
            return TerminalNode(terminal_info['value'], is_variable=False)
    
    # Se non abbiamo ancora raggiunto la profondità minima, creiamo solo nodi funzione
    if current_depth < min_depth:
        function_info = config.get_random_function()
        children = [grow_tree(config, max_depth, min_depth, current_depth + 1) for _ in range(function_info['arity'])]
        return FunctionNode(function_info['function'], function_info['arity'], function_info['symbol'], children)
    
    # Altrimenti, scegliamo casualmente tra funzioni e terminali
    if random.random() < 0.5:  # 50% di probabilità per funzioni o terminali
        function_info = config.get_random_function()
        children = [grow_tree(config, max_depth, min_depth, current_depth + 1) for _ in range(function_info['arity'])]
        return FunctionNode(function_info['function'], function_info['arity'], function_info['symbol'], children)
    else:
        terminal_info = config.get_random_terminal()
        if terminal_info['is_variable']:
            return TerminalNode(None, is_variable=True, var_index=terminal_info['var_index'])
        else:
            return TerminalNode(terminal_info['value'], is_variable=False)

In [14]:
def full_tree(config: GPConfig, max_depth: int, current_depth: int = 0) -> Node:
    """
    Metodo 'full' per generare un albero con tutti i rami alla stessa profondità
    
    Args:
        config: Configurazione del GP
        max_depth: Profondità massima dell'albero
        current_depth: Profondità corrente del nodo
        
    Returns:
        Nodo radice dell'albero generato
    """
    # Se siamo alla profondità massima, creiamo solo nodi terminali
    if current_depth >= max_depth:
        terminal_info = config.get_random_terminal()
        if terminal_info['is_variable']:
            return TerminalNode(None, is_variable=True, var_index=terminal_info['var_index'])
        else:
            return TerminalNode(terminal_info['value'], is_variable=False)
    
    # Altrimenti, creiamo solo nodi funzione
    function_info = config.get_random_function()
    children = [full_tree(config, max_depth, current_depth + 1) for _ in range(function_info['arity'])]
    return FunctionNode(function_info['function'], function_info['arity'], function_info['symbol'], children)


In [15]:
def ramped_half_and_half(config: GPConfig, min_depth: int, max_depth: int) -> ExpressionTree:
    """
    Metodo di inizializzazione 'ramped half-and-half'
    Combina 'grow' e 'full' per una maggiore diversità
    
    Args:
        config: Configurazione del GP
        min_depth: Profondità minima degli alberi
        max_depth: Profondità massima degli alberi
        
    Returns:
        Un nuovo albero di espressione
    """
    # Scegli una profondità casuale tra min_depth e max_depth
    depth = random.randint(min_depth, max_depth)
    
    # Scegli casualmente tra 'grow' e 'full'
    if random.random() < 0.5:
        root = grow_tree(config, depth, min_depth)
    else:
        root = full_tree(config, depth)
    
    return ExpressionTree(root)

In [16]:
def initialize_population(config: GPConfig) -> List[ExpressionTree]:
    """
    Crea la popolazione iniziale di alberi di espressione
    
    Args:
        config: Configurazione del GP
        
    Returns:
        Lista di alberi di espressione
    """
    population = []
    unique_expressions = set()  # Per il controllo dei duplicati
    
    print(f"Inizializzazione popolazione di {config.pop_size} individui...")
    start_time = time.time()
    
    # Continua a generare fino a quando non abbiamo abbastanza individui unici
    while len(population) < config.pop_size:
        tree = ramped_half_and_half(config, config.min_depth, config.max_depth)
        
        # Controllo dei duplicati (opzionale, ma aiuta la diversità iniziale)
        expr_str = tree.to_string()
        if expr_str not in unique_expressions:
            unique_expressions.add(expr_str)
            population.append(tree)
            
            # Aggiorniamo lo stato ogni 100 individui
            if len(population) % 1000 == 0:
                elapsed = time.time() - start_time
                print(f"  Generati {len(population)} individui in {elapsed:.2f} secondi")
    
    elapsed = time.time() - start_time
    print(f"Popolazione inizializzata in {elapsed:.2f} secondi")
    
    # Statistiche sulla popolazione iniziale
    heights = [tree.get_height() for tree in population]
    sizes = [tree.get_complexity() for tree in population]
    print(f"Statistiche sulla popolazione iniziale:")
    print(f"  - Altezza media: {np.mean(heights):.2f} (min: {min(heights)}, max: {max(heights)})")
    print(f"  - Dimensione media: {np.mean(sizes):.2f} nodi (min: {min(sizes)}, max: {max(sizes)})")
    
    return population

### Fitness Evaluation

In [17]:
def calculate_fitness(tree: ExpressionTree, X: np.ndarray, y: np.ndarray, 
                      parsimony_coef: float = 0.001) -> float:
    """
    Calcola il fitness di un individuo
    
    Args:
        tree: L'albero di espressione da valutare
        X: Input features
        y: Output target
        parsimony_coef: Coefficiente di penalità per la complessità dell'albero
        
    Returns:
        Valore di fitness (più basso è migliore)
    """
    try:
        # Valuta l'albero sui dati di input        
        predictions = tree.evaluate(X)        # Nel caso di valori NaN o infiniti, assegna un fitness molto alto (cattivo)
        if np.any(np.isnan(predictions)) or np.any(np.isinf(predictions)):
            return float('inf')
        
        # Calcola l'errore quadratico medio (MSE)
        mse = np.mean((predictions - y) ** 2)
        #Per il problema che stiamo affrontando, se la complessità è 1 (ovvero solo x[0] o x[1]),
        # aggiungiamo una piccola penalità per incoraggiare soluzioni più complesse
        #complexity = tree.get_complexity()
        #if complexity <= 1:
            # Penalizza leggermente le espressioni troppo semplici
        #    complexity_penalty = parsimony_coef * max(3 - complexity, 0)
        #else:
            # Penalizza normalmente la complessità per espressioni più grandi
        #    complexity_penalty = parsimony_coef * (complexity - 2)
        
        # Il fitness finale è MSE + penalità (più basso è migliore)
        #fitness = mse + complexity_penalty
        fitness = mse
        return fitness
    
    except Exception as e:
        # In caso di errori durante la valutazione, assegna un fitness molto alto
        print(f"Errore durante la valutazione: {e}")
        return float('inf')

In [18]:
def evaluate_population(population: List[ExpressionTree], X: np.ndarray, y: np.ndarray, 
                       config: GPConfig) -> None:
    """
    Valuta tutti gli individui della popolazione
    
    Args:
        population: Lista di alberi di espressione
        X: Input features
        y: Output target
        config: Configurazione del GP
    """
    for i, tree in enumerate(population):
        tree.fitness = calculate_fitness(tree, X, y, config.parsimony_coef)
    
 
   


In [19]:
def apply_fitness_sharing(population: List[ExpressionTree], sigma: float = 0.1) -> None:
    """
    Applica fitness sharing per favorire la diversità nella popolazione
    
    Args:
        population: Lista di alberi di espressione già valutati
        sigma: Raggio del kernel di sharing
    """
    n = len(population)
    
    # Prima calcola le distanze semantiche (o sintattiche)
    for i in range(n):
        # Inizializza il fattore di sharing
        sharing_factor = 1.0
        
        # Calcola la distanza rispetto a tutti gli altri individui
        for j in range(n):
            if i != j:
                # Qui si può usare una distanza sintattica o semantica
                # Per semplicità, usiamo la distanza basata sulla stringa dell'espressione
                expr_i = population[i].to_string()
                expr_j = population[j].to_string()
                
                # Calcola una misura di somiglianza (0 = completamente diversi, 1 = identici)
                # Si potrebbe usare una misura più sofisticata
                similarity = len(set(expr_i) & set(expr_j)) / len(set(expr_i) | set(expr_j))
                
                # Se la somiglianza è maggiore di una certa soglia, incrementa il fattore di sharing
                if similarity > 1 - sigma:
                    # Formula di sharing: 1 - (d/sigma)^2 se d < sigma, 0 altrimenti
                    d = 1 - similarity  # converte similarità in distanza
                    sharing_factor += max(0, 1 - (d/sigma)**2)
        
        # Evita la divisione per zero
        sharing_factor = min(10.0, max(1.0, sharing_factor))
        
        # Adjust fitness
        if population[i].fitness != float('inf'):
            population[i].adjusted_fitness = population[i].fitness * sharing_factor
        else:
            population[i].adjusted_fitness = float('inf')

In [20]:
def calculate_semantic_distance(tree1: ExpressionTree, tree2: ExpressionTree, 
                               X_sample: np.ndarray) -> float:
    """
    Calcola la distanza semantica tra due alberi di espressione
    basata sul comportamento su un campione di dati
    
    Args:
        tree1, tree2: Alberi di espressione da confrontare
        X_sample: Campione di dati di input per testare il comportamento
        
    Returns:
        Distanza semantica (0 = comportamento identico)
    """
    # Valutiamo gli alberi sul campione di dati
    try:
        output1 = tree1.evaluate(X_sample)
        output2 = tree2.evaluate(X_sample)
        
        # Calcolo della distanza (errore quadratico medio)
        if np.any(np.isnan(output1)) or np.any(np.isinf(output1)) or \
           np.any(np.isnan(output2)) or np.any(np.isinf(output2)):
            return float('inf')
        
        # Normalizzazione degli output per confronto più equo
        if np.std(output1) > 0 and np.std(output2) > 0:
            output1 = (output1 - np.mean(output1)) / np.std(output1)
            output2 = (output2 - np.mean(output2)) / np.std(output2)
        
        # Calcola distanza
        return np.mean((output1 - output2) ** 2)
    
    except Exception:
        # In caso di errori, consideriamo gli alberi molto distanti
        return float('inf')

### Selection Method

In [21]:
def tournament_selection(population: List[ExpressionTree], tournament_size: int, 
                         use_adjusted_fitness: bool = False) -> ExpressionTree:
    """
    Seleziona un individuo tramite selezione a torneo
    
    Args:
        population: Lista di alberi di espressione
        tournament_size: Numero di partecipanti al torneo
        use_adjusted_fitness: Se usare il fitness aggiustato per la diversità
        
    Returns:
        L'individuo selezionato
    """
    # Seleziona casualmente tournament_size individui dalla popolazione
    contestants = random.sample(population, min(tournament_size, len(population)))
    
    # Trova l'individuo con il miglior fitness (il più basso)
    if use_adjusted_fitness:
        # Usa il fitness aggiustato per la diversità, se disponibile
        best = min(contestants, key=lambda x: float('inf') if x.adjusted_fitness is None else x.adjusted_fitness)
    else:
        best = min(contestants, key=lambda x: float('inf') if x.fitness is None else x.fitness)
    
    return best


In [22]:
def age_weighted_selection(population: List[ExpressionTree], 
                          max_age: int = 10, 
                          young_advantage: float = 0.3) -> ExpressionTree:
    """
    Selezione che favorisce individui più giovani (con minore età)
    
    Args:
        population: Lista di alberi di espressione
        max_age: Età massima considerata per il vantaggio
        young_advantage: Vantaggio percentuale per individui giovani
        
    Returns:
        L'individuo selezionato
    """
    # Calcola pesi basati sull'età
    age_weights = [max(0.1, 1.0 - (tree.age / max_age) * young_advantage) 
                  for tree in population]
    
    # Normalizza i pesi
    total_weight = sum(age_weights)
    if total_weight > 0:
        norm_weights = [w / total_weight for w in age_weights]
    else:
        norm_weights = [1.0 / len(population)] * len(population)
    
    # Selezione pesata
    return random.choices(population, weights=norm_weights, k=1)[0]


In [23]:
def diversity_tournament_selection(population: List[ExpressionTree], 
                                  X_sample: np.ndarray,
                                  tournament_size: int = 5, 
                                  diversity_weight: float = 0.3) -> ExpressionTree:
    """
    Selezione a torneo che considera sia il fitness che la diversità semantica
    
    Args:
        population: Lista di alberi di espressione
        X_sample: Campione di dati per calcolare il comportamento semantico
        tournament_size: Numero di partecipanti al torneo
        diversity_weight: Peso dato alla diversità (vs. fitness)
        
    Returns:
        L'individuo selezionato
    """
    if len(population) <= 1:
        return population[0]
    
    # Seleziona casualmente tournament_size individui
    contestants = random.sample(population, min(tournament_size, len(population)))
    
    # Trova i punteggi di fitness (normalizzati)
    fitness_values = [tree.fitness for tree in contestants]
    max_fitness = max(fitness_values)
    min_fitness = min(fitness_values)
    fitness_range = max_fitness - min_fitness
    
    if fitness_range > 0:
        norm_fitness = [(f - min_fitness) / fitness_range for f in fitness_values]
    else:
        norm_fitness = [0.5] * len(contestants)
    
    # Calcola la diversità di ogni individuo rispetto alla popolazione
    diversity_scores = []
    
    for i, tree in enumerate(contestants):
        diversity = 0
        valid_comparisons = 0
        
        # Confronta con un campione casuale della popolazione per efficienza
        population_sample = random.sample(population, min(10, len(population)))
        
        for other in population_sample:
            if tree != other:
                dist = calculate_semantic_distance(tree, other, X_sample)
                if dist != float('inf'):
                    diversity += dist
                    valid_comparisons += 1
        
        # Media delle distanze o valore predefinito se non ci sono confronti validi
        if valid_comparisons > 0:
            diversity_scores.append(diversity / valid_comparisons)
        else:
            diversity_scores.append(0.0)
    
    # Normalizza i punteggi di diversità
    max_div = max(diversity_scores) if diversity_scores else 1.0
    min_div = min(diversity_scores) if diversity_scores else 0.0
    div_range = max_div - min_div
    
    if div_range > 0:
        norm_diversity = [(d - min_div) / div_range for d in diversity_scores]
    else:
        norm_diversity = [0.5] * len(contestants)
    
    # Combina fitness e diversità per un punteggio totale
    # Più alto è migliore (quindi invertiamo il fitness normalizzato)
    total_scores = [(1 - nf) * (1 - diversity_weight) + nd * diversity_weight 
                   for nf, nd in zip(norm_fitness, norm_diversity)]
    
    # Seleziona l'individuo con il miglior punteggio totale
    best_idx = total_scores.index(max(total_scores))
    return contestants[best_idx]

In [24]:
def select_parents(population: List[ExpressionTree], config: GPConfig, 
                  X_sample: np.ndarray) -> Tuple[ExpressionTree, ExpressionTree]:
    """
    Seleziona due genitori dalla popolazione
    
    Args:
        population: Lista di alberi di espressione
        config: Configurazione del GP
        X_sample: Campione di dati per calcolare la diversità semantica
        
    Returns:
        Coppia di genitori
    """
    # Scegliamo casualmente il metodo di selezione
    selection_r = random.random()
    
    if selection_r < 0.9:  # 90% probabilità di usare il torneo standard con adjusted fitness
        parent1 = tournament_selection(population, config.tournament_size, use_adjusted_fitness=True)
        parent2 = tournament_selection(population, config.tournament_size, use_adjusted_fitness=True)
    else:  # 10% probabilità di usare selezione basata sull'età
        parent1 = age_weighted_selection(population)
        parent2 = age_weighted_selection(population)
    
    # Assicurati che i genitori siano diversi
    attempts = 0
    while parent1 == parent2 and attempts < 5:
        parent2 = tournament_selection(population, config.tournament_size)
        attempts += 1
    
    return parent1, parent2

### Genetic Operator

In [25]:
def subtree_crossover(parent1: ExpressionTree, parent2: ExpressionTree, 
                     max_tries: int = 5, max_depth: int = 10) -> Tuple[ExpressionTree, ExpressionTree]:
    """
    Crossover per alberi: scambia sottoalberi tra i genitori
    
    Args:
        parent1, parent2: Alberi genitori
        max_tries: Numero massimo di tentativi per trovare punti di crossover validi
        max_depth: Profondità massima consentita per l'albero risultante
        
    Returns:
        Due alberi figli generati dal crossover
    """
    # Creiamo copie dei genitori
    child1 = parent1.copy()
    child2 = parent2.copy()
    
    # Ottieni tutti i nodi negli alberi
    nodes1 = child1.get_nodes()
    nodes2 = child2.get_nodes()
    
    if not nodes1 or not nodes2:
        return child1, child2  # Non possiamo fare crossover se uno degli alberi è vuoto
    
    # Tenta il crossover per un numero limitato di volte
    for _ in range(max_tries):
        # Scegli casualmente i punti di crossover
        crossover_point1 = random.randrange(len(nodes1))
        crossover_point2 = random.randrange(len(nodes2))
        
        # Ottieni i sottoalberi
        subtree1 = nodes1[crossover_point1]
        subtree2 = nodes2[crossover_point2]
        
        # Crea copie dei sottoalberi
        subtree1_copy = subtree1.copy()
        subtree2_copy = subtree2.copy()
        
        # Sostituisci i sottoalberi
        child1.replace_subtree_at_index(crossover_point1, subtree2_copy)
        child2.replace_subtree_at_index(crossover_point2, subtree1_copy)
        
        # Aggiorna le profondità dei nodi
        child1.update_node_depths()
        child2.update_node_depths()
        
        # Controlla se i figli rispettano il vincolo di profondità massima
        if child1.get_height() <= max_depth and child2.get_height() <= max_depth:
            # Crossover riuscito
            break
        else:
            # Ripristina i figli dalle copie dei genitori
            child1 = parent1.copy()
            child2 = parent2.copy()
            nodes1 = child1.get_nodes()
            nodes2 = child2.get_nodes()
    
    # Incrementa l'età
    child1.age = 0
    child2.age = 0
    
    return child1, child2

In [26]:
def subtree_mutation(tree: ExpressionTree, config: GPConfig, 
                    max_depth: int = 10) -> ExpressionTree:
    """
    Mutazione di sottoalbero: sostituisce un sottoalbero casuale con uno nuovo
    
    Args:
        tree: Albero da mutare
        config: Configurazione del GP
        max_depth: Profondità massima consentita per l'albero risultante
        
    Returns:
        Albero mutato
    """
    # Crea una copia dell'albero
    mutated = tree.copy()
    
    # Ottieni tutti i nodi nell'albero
    nodes = mutated.get_nodes()
    
    if not nodes:
        return mutated  # Non possiamo mutare un albero vuoto
    
    # Scegli casualmente un punto di mutazione
    mutation_point = random.randrange(len(nodes))
    
    # Calcola la profondità massima per il nuovo sottoalbero
    node_depth = nodes[mutation_point].depth
    remaining_depth = max_depth - node_depth
    
    if remaining_depth < 1:
        return mutated  # Non possiamo mutare se non c'è spazio per crescere
    
    # Genera un nuovo sottoalbero casuale
    new_subtree = grow_tree(config, remaining_depth, min_depth=1)
    
    # Sostituisci il sottoalbero
    mutated.replace_subtree_at_index(mutation_point, new_subtree)
    
    # Aggiorna le profondità dei nodi
    mutated.update_node_depths()
    
    # Resetta l'età
    mutated.age = 0
    
    return mutated

In [27]:
def point_mutation(tree: ExpressionTree, config: GPConfig) -> ExpressionTree:
    """
    Mutazione puntuale: cambia un singolo nodo mantenendo la struttura dell'albero
    
    Args:
        tree: Albero da mutare
        config: Configurazione del GP
        
    Returns:
        Albero mutato
    """
    # Crea una copia dell'albero
    mutated = tree.copy()
    
    # Ottieni tutti i nodi nell'albero
    nodes = mutated.get_nodes()
    
    if not nodes:
        return mutated  # Non possiamo mutare un albero vuoto
    
    # Scegli casualmente un punto di mutazione
    mutation_point = random.randrange(len(nodes))
    node = nodes[mutation_point]
    
    # Mutazione basata sul tipo di nodo
    if isinstance(node, FunctionNode):
        # Sostituisci con un'altra funzione della stessa arità
        compatible_functions = [f for f in config.function_set if f['arity'] == node.arity]
        if compatible_functions:
            function_info = random.choice(compatible_functions)
            new_node = FunctionNode(function_info['function'], 
                                   function_info['arity'], 
                                   function_info['symbol'],
                                   node.children.copy())  # riutilizza gli stessi figli
            
            # Sostituisci il nodo
            mutated.replace_subtree_at_index(mutation_point, new_node)
    
    elif isinstance(node, TerminalNode):
        if node.is_variable:
            # Sostituisci con un'altra variabile
            if len(config.variable_terminals) > 1:
                terminal_info = config.get_random_variable()
                while terminal_info['var_index'] == node.var_index:
                    terminal_info = config.get_random_variable()
                
                new_node = TerminalNode(None, True, terminal_info['var_index'])
                mutated.replace_subtree_at_index(mutation_point, new_node)
        else:
            # Potremmo sostituire con un'altra costante o modificare leggermente il valore
            if random.random() < 0.5:  # 50% probabilità di modificare il valore
                # Modifica il valore esistente (small perturbation)
                new_value = node.value * (1.0 + random.uniform(-0.1, 0.1))
                new_node = TerminalNode(new_value, False)
            else:
                # Sostituisci con una nuova costante
                terminal_info = config.get_random_constant()
                new_node = TerminalNode(terminal_info['value'], False)
            
            mutated.replace_subtree_at_index(mutation_point, new_node)
    
    # Aggiorna le profondità dei nodi
    mutated.update_node_depths()
    
    # Resetta l'età
    mutated.age = 0
    
    return mutated


In [28]:
def deterministic_crowding(parent1: ExpressionTree, parent2: ExpressionTree,
                          child1: ExpressionTree, child2: ExpressionTree,
                          X: np.ndarray, y: np.ndarray, config: GPConfig) -> List[ExpressionTree]:
    """
    Deterministic crowding: i figli sostituiscono i genitori solo se hanno fitness migliore
    
    Args:
        parent1, parent2: Genitori
        child1, child2: Figli generati dai genitori
        X, y: Dati per la valutazione
        config: Configurazione del GP
        
    Returns:
        Lista di individui selezionati
    """
    # Calcola il fitness dei figli
    child1.fitness = calculate_fitness(child1, X, y, config.parsimony_coef)
    child2.fitness = calculate_fitness(child2, X, y, config.parsimony_coef)
    
    # Applica fitness sharing ai figli
    # Possiamo creare una piccola popolazione temporanea con i figli per calcolare il loro adjusted_fitness
    temp_pop = [parent1, parent2, child1, child2]
    apply_fitness_sharing(temp_pop)

    # Calcola similarità tra genitori e figli
    X_sample = X[:min(len(X), 100)]  # Usa un campione per efficienza
    
    dist_p1c1 = calculate_semantic_distance(parent1, child1, X_sample)
    dist_p1c2 = calculate_semantic_distance(parent1, child2, X_sample)
    
    # Decidi quali abbinamenti genitore-figlio confrontare
    if dist_p1c1 <= dist_p1c2:
        # parent1 vs child1, parent2 vs child2
        competition1 = (parent1, child1)
        competition2 = (parent2, child2)
    else:
        # parent1 vs child2, parent2 vs child1
        competition1 = (parent1, child2)
        competition2 = (parent2, child1)
    
    result = []
    
    # Prima competizione
    if competition1[1].adjusted_fitness <= competition1[0].adjusted_fitness:
        result.append(competition1[1])  # Il figlio vince
    else:
        result.append(competition1[0])  # Il genitore vince
    
    # Seconda competizione
    if competition2[1].adjusted_fitness <= competition2[0].adjusted_fitness:
        result.append(competition2[1])  # Il figlio vince
    else:
        result.append(competition2[0])  # Il genitore vince
    
    return result

###  Genetic Operator

In [29]:
def apply_genetic_operators(population: List[ExpressionTree], X: np.ndarray, y: np.ndarray, 
                           config: GPConfig) -> List[ExpressionTree]:
    """
    Applica operatori genetici per creare una nuova popolazione
    
    Args:
        population: Lista di alberi di espressione
        X, y: Dati per la valutazione
        config: Configurazione del GP
        
    Returns:
        Nuova popolazione
    """
    # Ordina la popolazione per fitness (migliore al primo posto)
    sorted_population = sorted(population, key=lambda x: float('inf') if x.adjusted_fitness is None else x.adjusted_fitness)
    
    # Numero di individui da selezionare tramite elitismo
    n_elite = int(config.pop_size * config.elitism_rate)
    
    # Campione per calcolare diversità semantica
    X_sample = X[:min(len(X), 100)]  # Usa un campione per efficienza
    
    # Seleziona gli elite
    new_population = [tree.copy() for tree in sorted_population[:n_elite]]
    
    # Incrementa l'età di ogni individuo elite
    for tree in new_population:
        tree.age += 1
    
    # Completa la popolazione con nuovi individui
    while len(new_population) < config.pop_size:
        # Seleziona operazione genetica (crossover o mutazione)
        op_choice = random.random()
        
        if op_choice < config.crossover_prob:
            # Crossover
            parent1, parent2 = select_parents(population, config, X_sample)
            child1, child2 = subtree_crossover(parent1, parent2, max_depth=config.max_depth)
            
            # Usa deterministic crowding per decidere quali individui tenere
            selected = deterministic_crowding(parent1, parent2, child1, child2, X, y, config)
            
            # Aggiungi alla nuova popolazione
            new_population.extend(selected)
            if len(new_population) > config.pop_size:
                new_population = new_population[:config.pop_size]
        
        elif op_choice < config.crossover_prob + config.mutation_prob:
            # Mutazione
            parent = tournament_selection(population, config.tournament_size)
            
            # Scegli casualmente il tipo di mutazione
            mutation_choice = random.random()
            
            if mutation_choice < 0.7:  # 70% subtree mutation
                child = subtree_mutation(parent, config, max_depth=config.max_depth)
            else:  # 30% point mutation
                child = point_mutation(parent, config)
            
            # Calcola fitness del figlio
            child.fitness = calculate_fitness(child, X, y, config.parsimony_coef)
            temp_pop = [parent, child]
            apply_fitness_sharing(temp_pop)
            # Confronta genitore e figlio
            if child.adjusted_fitness <= parent.adjusted_fitness:
                new_population.append(child)
            else:
                # Aggiungi comunque il figlio con una certa probabilità
                if random.random() < 0.1:  # 10% di probabilità
                    new_population.append(child)
                else:
                    new_population.append(parent.copy())
        
        else:
            # Riproduzione (copia diretta)
            parent = tournament_selection(population, config.tournament_size)
            offspring = parent.copy()
            offspring.age += 1  # Incrementa l'età
            new_population.append(offspring)
    
    # Assicurati che la popolazione abbia esattamente la dimensione richiesta
    if len(new_population) > config.pop_size:
        new_population = new_population[:config.pop_size]
    
    # Aggiorna il fitness adjusted per diversità
    apply_fitness_sharing(new_population)
    
    return new_population

### Diversity Maintenance

In [30]:
class Island:
    """Rappresenta un'isola nell'Island Model"""
    
    def __init__(self, population: List[ExpressionTree], config: GPConfig, island_id: int):
        self.population = population
        self.config = config
        self.id = island_id
        self.best_individual = None
        self.best_fitness = float('inf')
        self.generations_without_improvement = 0
    
    def evolve(self, X: np.ndarray, y: np.ndarray, generation: int) -> None:
        """
        Evolve l'isola per una generazione
        
        Args:
            X, y: Dati di training
            generation: Numero della generazione corrente
        """
        # Applica operatori genetici
        self.population = apply_genetic_operators(self.population, X, y, self.config)

         # Valuta la popolazione
        #evaluate_population(self.population, X, y, self.config) 

        # Applica fitness sharing per mantenere la diversità
        #apply_fitness_sharing(self.population)
        
        # Aggiorna la miglior soluzione
        current_best = min(self.population, key=lambda x: float('inf') if x.adjusted_fitness is None else x.adjusted_fitness)
        if current_best.adjusted_fitness < self.best_fitness:
            self.best_individual = current_best.copy()
            self.best_fitness = current_best.adjusted_fitness
            self.generations_without_improvement = 0
        else:
            self.generations_without_improvement += 1
        
        # Log dei progressi
        if generation % 5 == 0:  # Ogni 5 generazioni
            ic(f"Isola {self.id}", f"Gen {generation}", self.best_fitness)

In [31]:
def initialize_islands(total_population: List[ExpressionTree], config: GPConfig, n_islands: int = 3) -> List[Island]:
    """
    Inizializza le isole dividendo la popolazione
    
    Args:
        total_population: Lista completa di alberi di espressione
        config: Configurazione del GP
        n_islands: Numero di isole da creare
        
    Returns:
        Lista di oggetti Island
    """
    islands = []
    # Mescola casualmente la popolazione
    shuffled_population = random.sample(total_population, len(total_population))
    
    # Calcola la dimensione di ciascuna isola
    island_size = len(shuffled_population ) // n_islands
    
    # Distribuisci la popolazione tra le isole
    for i in range(n_islands):
        start_idx = i * island_size
        end_idx = start_idx + island_size if i < n_islands - 1 else len(shuffled_population)
        island_population = shuffled_population[start_idx:end_idx]
        
        # Crea configurazione specifica per l'isola
        island_config = copy.deepcopy(config)
        island_config.pop_size = len(island_population)
        
        # Crea l'isola
        island = Island(island_population, island_config, i)
        islands.append(island)
    
    return islands


In [32]:
def migration(islands: List[Island], migration_rate: float = 0.2) -> None:
    """
    Permette la migrazione di individui tra isole
    
    Args:
        islands: Lista di oggetti Island
        migration_rate: Percentuale di popolazione che migra
    """
    if len(islands) <= 1:
        return
    
    print("Esecuzione migrazione tra isole...")
    
    for i, source_island in enumerate(islands):
        # Calcola l'isola di destinazione (la successiva, o la prima se è l'ultima)
        dest_idx = (i + 1) % len(islands)
        dest_island = islands[dest_idx]
        
        # Numero di individui da migrare
        n_migrants = max(1, int(source_island.config.pop_size * migration_rate))
        
        # Seleziona i migranti (metà migliori, metà casuali)
        n_best = n_migrants // 2
        n_random = n_migrants - n_best
        
        # Ordina per fitness
        sorted_pop = sorted(source_island.population, 
                          key=lambda x: float('inf') if x.adjusted_fitness is None else x.adjusted_fitness)
        
        # Prendi i migliori
        migrants_best = [ind.copy() for ind in sorted_pop[:n_best]]
        
        # Prendi alcuni casuali
        migrants_random = [ind.copy() for ind in random.sample(source_island.population, n_random)]
        
        migrants = migrants_best + migrants_random
        
        # Sostituisci i peggiori nell'isola di destinazione
        dest_sorted = sorted(dest_island.population, 
                           key=lambda x: float('inf') if x.adjusted_fitness is None else x.adjusted_fitness, 
                           reverse=True)  # ordine decrescente
        
        # Rimuovi i peggiori
        for j in range(min(n_migrants, len(dest_sorted))):
            dest_island.population.remove(dest_sorted[j])
        
        # Aggiungi i migranti
        dest_island.population.extend(migrants)
        
        print(f"  Migrazione: {n_migrants} individui dall'isola {i} all'isola {dest_idx}")

In [33]:
def apply_novelty_search(population: List[ExpressionTree], X: np.ndarray, 
                        archive_size: int = 10) -> None:
    """
    Applica novelty search per promuovere la diversità
    
    Args:
        population: Lista di alberi di espressione
        X: Dati di input per calcolare il comportamento
        archive_size: Dimensione dell'archivio delle novità
    """
    # Campione per calcolare il comportamento
    X_sample = X[:min(len(X), 100)]
    
    # Calcola il "comportamento" di ogni individuo
    behaviors = []
    for tree in population:
        try:
            # Usiamo l'output su X_sample come comportamento
            behavior = tree.evaluate(X_sample)
            behaviors.append(behavior)
        except:
            # In caso di errori usiamo un vettore di zeri
            behaviors.append(np.zeros(len(X_sample)))
    
    # Calcola la novelty score per ogni individuo
    k = min(5, len(population) - 1)  # numero di vicini più prossimi
    
    for i, tree in enumerate(population):
        # Calcola distanze rispetto a tutti gli altri
        distances = []
        for j, other_behavior in enumerate(behaviors):
            if i != j:
                # Distanza del comportamento
                dist = np.mean((behaviors[i] - other_behavior) ** 2)
                distances.append(dist)
        
        # Calcola la novità come media delle k distanze più piccole
        if distances:
            distances.sort()
            novelty = np.mean(distances[:k])
            # Aggiungi la novità al fitness adjusted
            if tree.adjusted_fitness is not None and tree.adjusted_fitness != float('inf'):
                # Combina fitness e novità (più basso è meglio per fitness, più alto è meglio per novità)
                tree.adjusted_fitness = tree.adjusted_fitness * (1.0 - 0.2 / (1.0 + novelty))

### Bloat Control

In [34]:
def apply_bloat_control(population: List[ExpressionTree], config: GPConfig) -> None:
    """
    Applica varie tecniche di controllo del bloat
    
    Args:
        population: Lista di alberi di espressione
        config: Configurazione del GP
    """
    # Controllo della dimensione massima
    oversized = [i for i, tree in enumerate(population) if tree.get_complexity() > config.max_tree_size]
    
    if oversized:
        print(f"Controllo del bloat: {len(oversized)} individui superano la dimensione massima")
        
        for idx in oversized:
            # Tenta di ridurre la dimensione sostituendo sottoalberi con terminali
            tree = population[idx]
            nodes = tree.get_nodes()
            
            # Trova nodi non terminali a profondità maggiore
            non_terminal_nodes = [i for i, node in enumerate(nodes) 
                                if isinstance(node, FunctionNode) and node.depth > 2]
            
            if non_terminal_nodes:
                # Sostituisci un nodo casuale con un terminale
                node_idx = random.choice(non_terminal_nodes)
                
                # Crea un nuovo nodo terminale
                terminal_info = config.get_random_terminal()
                if terminal_info['is_variable']:
                    new_node = TerminalNode(None, True, terminal_info['var_index'])
                else:
                    new_node = TerminalNode(terminal_info['value'], False)
                
                # Sostituisci il sottoalbero
                tree.replace_subtree_at_index(node_idx, new_node)
                tree.update_node_depths()
    
    # Applica lexicographic parsimony pressure
    # Quando due individui hanno fitness simili, preferisce quello più semplice
    epsilon = 0.0000001  # soglia per considerare fitness simili
    
    for i in range(len(population)):
        for j in range(i + 1, len(population)):
            tree_i = population[i]
            tree_j = population[j]
            
            # Se i fitness sono simili
            if abs(tree_i.adjusted_fitness - tree_j.adjusted_fitness) < epsilon:
                # Ottieni complessità
                complexity_i = tree_i.get_complexity()
                complexity_j = tree_j.get_complexity()
                
                # Se l'albero j è significativamente più complesso, penalizzarlo
                if complexity_j > complexity_i * 1.5:
                    # Aggiungi una piccola penalità al fitness
                    tree_j.adjusted_fitness += epsilon
                
                # Se l'albero i è significativamente più complesso, penalizzarlo
                elif complexity_i > complexity_j * 1.5:
                    # Aggiungi una piccola penalità al fitness
                    tree_i.adjusted_fitness += epsilon

### Princial Algorithm

In [35]:
def genetic_programming(X: np.ndarray, y: np.ndarray, config: GPConfig, 
                       use_islands: bool = False, n_islands: int = 5, 
                       migration_interval: int = 100) -> ExpressionTree:
    """
    Algoritmo principale di Genetic Programming per Symbolic Regression
    
    Args:
        X: Input features
        y: Output target
        config: Configurazione del GP
        use_islands: Se usare il modello a isole
        n_islands: Numero di isole (se use_islands è True)
        migration_interval: Intervallo di generazioni tra migrazioni
        
    Returns:
        Il miglior albero di espressione trovato
    """
    start_time = time.time()
    print(f"Avvio Genetic Programming per Symbolic Regression...")
    print(f"Configurazione: pop_size={config.pop_size}, max_depth={config.max_depth}, "
          f"generations={config.generations}")
    
    # Inizializza la popolazione
    initial_population = initialize_population(config)
    

    # Valuta la popolazione iniziale
    evaluate_population(initial_population, X, y, config)
    
    # Applica fitness sharing per diversità
    apply_fitness_sharing(initial_population) 
    
    # Inizializza isole o popolazione unica
    if use_islands:
        islands = initialize_islands(initial_population, config, n_islands)
        best_individual = min([island.best_individual for island in islands if island.best_individual], 
                            key=lambda x: x.adjusted_fitness, default=None)
        best_fitness = float('inf') if best_individual is None else best_individual.adjusted_fitness
    else:
        population = initial_population
        best_individual = min(population, key=lambda x: float('inf') if x.adjusted_fitness is None else x.adjusted_fitness)
        best_fitness = best_individual.adjusted_fitness
    
    # Statistiche per monitoraggio
    stats = {
        'best_fitness': [],
        'avg_fitness': [],
        'avg_size': [],
        'best_size': [],
        'diversity': []
    }
    
    # Loop principale evolutivo
    generations_without_improvement = 0
    for generation in tqdm(range(config.generations)):
        generation_start = time.time()
        
        if use_islands:
            # Evolvi ogni isola separatamente
            for island in islands:
                island.evolve(X, y, generation)
            
            # Migrazione periodica
            if (generation + 1) % migration_interval == 0:
                migration(islands, migration_rate=0.1)
            
            # Calcola il miglior individuo globale
            current_best = min([island.best_individual for island in islands if island.best_individual], 
                             key=lambda x: x.adjusted_fitness)
            
            # Calcola statistiche aggregate
            all_individuals = []
            for island in islands:
                all_individuals.extend(island.population)
            
            avg_fitness = np.mean([tree.adjusted_fitness for tree in all_individuals if tree.adjusted_fitness != float('inf')])
            avg_size = np.mean([tree.get_complexity() for tree in all_individuals])
            
            # Calcola diversità (numero di espressioni uniche)
            unique_expressions = set()
            for tree in all_individuals:
                unique_expressions.add(tree.to_string())
            diversity = len(unique_expressions) / len(all_individuals)
            
        else:
            # Applica operatori genetici
            population = apply_genetic_operators(population, X, y, config)
            
            #apply_novelty_search(population, X)
            
            # Controllo del bloat
            apply_bloat_control(population, config)
            
            # Calcola il miglior individuo
            current_best = min(population, key=lambda x: float('inf') if x.adjusted_fitness is None else x.adjusted_fitness)
            
            # Calcola statistiche
            avg_fitness = np.mean([tree.adjusted_fitness for tree in population if tree.adjusted_fitness != float('inf')])
            avg_size = np.mean([tree.get_complexity() for tree in population])
            
            # Calcola diversità (numero di espressioni uniche)
            unique_expressions = set()
            for tree in population:
                unique_expressions.add(tree.to_string())
            diversity = len(unique_expressions) / len(population)
        
        # Aggiorna il miglior individuo globale
        if current_best.adjusted_fitness < best_fitness:
            best_individual = current_best.copy()
            best_fitness = current_best.adjusted_fitness
            generations_without_improvement = 0
            print(f"Nuova miglior soluzione trovata:")
            print(f"  Espressione: {best_individual.to_string()}")
            print(f"  Fitness: {best_fitness}")
            print(f"  Complessità: {best_individual.get_complexity()} nodi")
        else:
            generations_without_improvement += 1
        
        # Registra statistiche
        stats['best_fitness'].append(best_fitness)
        stats['avg_fitness'].append(avg_fitness)
        stats['avg_size'].append(avg_size)
        stats['best_size'].append(best_individual.get_complexity())
        stats['diversity'].append(diversity)
        
        # Log della generazione
        generation_time = time.time() - generation_start
        if generation % 5 == 0 or generation == config.generations - 1:
           ic(f"Generazione {generation}", best_fitness, f"Diversità {diversity:.2f}", f"Tempo {generation_time:.2f}s")
        
        # Criterio di terminazione anticipata
        if generations_without_improvement >= 100:
            print("Terminazione anticipata: nessun miglioramento nelle ultime 20 generazioni")
            break
    
    total_time = time.time() - start_time
    print(f"Algoritmo completato in {total_time:.2f} secondi")
    print(f"Miglior soluzione trovata:")
    print(f"  Espressione: {best_individual.to_string()}")
    print(f"  Fitness: {best_fitness}")
    print(f"  Complessità: {best_individual.get_complexity()} nodi")
    
    # Visualizza statistiche
    plot_statistics(stats)
    
    return best_individual

### Termination AND Evaluetion

In [36]:

def plot_statistics(stats: Dict) -> None:
    """
    Visualizza le statistiche dell'esecuzione
    
    Args:
        stats: Dizionario con le statistiche raccolte
    """
    fig, axs = plt.subplots(3, 1, figsize=(10, 12))
    
    # Fitness plot
    axs[0].plot(stats['best_fitness'], label='Miglior fitness')
    axs[0].plot(stats['avg_fitness'], label='Fitness medio')
    axs[0].set_title('Evoluzione del fitness')
    axs[0].set_xlabel('Generazione')
    axs[0].set_ylabel('Fitness (MSE)')
    axs[0].legend()
    axs[0].grid(True)
    
    # Size plot
    axs[1].plot(stats['best_size'], label='Dimensione miglior individuo')
    axs[1].plot(stats['avg_size'], label='Dimensione media')
    axs[1].set_title('Evoluzione della dimensione')
    axs[1].set_xlabel('Generazione')
    axs[1].set_ylabel('Numero di nodi')
    axs[1].legend()
    axs[1].grid(True)
    
    # Diversity plot
    axs[2].plot(stats['diversity'], label='Diversità')
    axs[2].set_title('Evoluzione della diversità')
    axs[2].set_xlabel('Generazione')
    axs[2].set_ylabel('Proporzione di espressioni uniche')
    axs[2].legend()
    axs[2].grid(True)
    
    plt.tight_layout()
    plt.savefig('gp_statistics.png')
    plt.show()


def plot_predictions(tree: ExpressionTree, X: np.ndarray, y: np.ndarray, title: str) -> None:
    """
    Visualizza le previsioni del modello confrontate con i dati reali
    
    Args:
        tree: Albero di espressione
        X: Input features
        y: Output target
        title: Titolo del grafico
    """
    # Calcola le previsioni
    predictions = tree.evaluate(X)
    
    plt.figure(figsize=(10, 6))
    
    if X.ndim == 1 or (X.ndim == 2 and X.shape[1] == 1):
        # Per dati 1D, visualizza previsioni vs. dati reali
        x_plot = X.flatten() if X.ndim == 2 else X
        sort_idx = np.argsort(x_plot)
        x_plot = x_plot[sort_idx]
        y_plot = y[sort_idx]
        predictions = predictions[sort_idx]
        
        plt.scatter(x_plot, y_plot, alpha=0.5, label='Dati reali')
        plt.plot(x_plot, predictions, 'r-', linewidth=2, label='Modello GP')
        plt.xlabel('X')
        plt.ylabel('y')
    else:
        # Per dati multidimensionali, visualizza previsioni vs. reali
        plt.scatter(y, predictions, alpha=0.5)
        plt.plot([min(y), max(y)], [min(y), max(y)], 'r--', linewidth=2)
        plt.xlabel('Target reale')
        plt.ylabel('Previsione')
    
    plt.title(title)
    plt.legend()
    plt.grid(True)
    plt.savefig('gp_predictions.png')
    plt.show()

In [37]:
def simplify_expression(expression: str) -> str:
    """
    Semplifica un'espressione (implementazione base)
    
    Args:
        expression: Espressione come stringa
        
    Returns:
        Espressione semplificata
    """
    # Questo è un semplificatore molto elementare
    # Per una semplificazione completa sarebbe necessario utilizzare una libreria
    # come sympy
    
    # Esempio di alcune semplificazioni
    simplifications = [
        # Operazioni con 0
        ('(0.0000 + ', '('),
        ('(0.0000 * ', '(0.0000 * '),
        ('+ 0.0000)', ')'),
        ('* 0.0000)', '* 0.0000)'),
        ('(x[0] * 0.0000)', '0.0000'),
        ('(0.0000 * x[0])', '0.0000'),
        
        # Operazioni con 1
        ('(x[0] * 1.0000)', 'x[0]'),
        ('(1.0000 * x[0])', 'x[0]'),
        ('(x[0] / 1.0000)', 'x[0]'),
        
        # Operazioni con se stesso
        ('(x[0] - x[0])', '0.0000'),
        ('(x[0] / x[0])', '1.0000'),
    ]
    
    # Applica semplificazioni ripetutamente
    prev_expr = ""
    simplified = expression
    while prev_expr != simplified:
        prev_expr = simplified
        for pattern, replacement in simplifications:
            simplified = simplified.replace(pattern, replacement)
    
    return simplified

### Configuration and Execution 

In [38]:
def run_gp_on_problem(file_path: str, config_overrides: Dict = None) -> ExpressionTree:
    """
    Esegue l'algoritmo GP su un problema specifico
    
    Args:
        file_path: Percorso del file del problema
        config_overrides: Sovrascritture alla configurazione predefinita
        
    Returns:
        Il miglior albero di espressione trovato
    """
    # Carica e prepara i dati
    X, y, data_config = prepare_data(file_path)
    
    # Crea la configurazione base
    config = GPConfig(
        n_features=data_config['n_features'],
        const_range=data_config['const_range'],
        use_trig=True,
        use_exp_log=True,
        min_depth=2,
        max_depth=6,
        pop_size=500,
        generations=50,
        tournament_size=100,
        crossover_prob=0.7,
        mutation_prob=0.05,
        elitism_rate=0.1,
        max_tree_size=50,
        parsimony_coef=0.01,
        diversity_weight=0.2
    )
    
    # Applica sovrascritture alla configurazione
    if config_overrides:
        for key, value in config_overrides.items():
            if hasattr(config, key):
                setattr(config, key, value)
    
    # Esegui l'algoritmo
    print(f"\nEsecuzione GP su {file_path}...")
    best_tree = genetic_programming(X, y, config, use_islands=True)
    
    # Visualizza risultati
    plot_predictions(best_tree, X, y, f"Previsioni GP su {file_path}")
    
    # Semplifica l'espressione
    simplified_expr = simplify_expression(best_tree.to_string())
    print(f"Espressione semplificata: {simplified_expr}")
    
    return best_tree


In [39]:
problems=[
    {"file_path": "../data/problem_0.npz", "config":{"max_depth": 3, "pop_size": 10000, "generations": 1000}},
    {"file_path": "../data/problem_1.npz", "config":{"max_depth": 8, "pop_size": 10000, "generations": 1000}},
    {"file_path": "../data/problem_2.npz", "config":{"max_depth": 8, "pop_size": 10000, "generations": 1000}},
    {"file_path": "../data/problem_3.npz", "config":{"max_depth": 8, "pop_size": 10000, "generations": 1000}},
    {"file_path": "../data/problem_4.npz", "config":{"max_depth": 8, "pop_size": 10000, "generations": 1000}},
    {"file_path": "../data/problem_5.npz", "config":{"max_depth": 8, "pop_size": 10000, "generations": 1000}},
    {"file_path": "../data/problem_6.npz", "config":{"max_depth": 8, "pop_size": 10000, "generations": 1000}},
    {"file_path": "../data/problem_7.npz", "config":{"max_depth": 8, "pop_size": 10000, "generations": 1000}},
    {"file_path": "../data/problem_8.npz", "config":{"max_depth": 8, "pop_size": 10000, "generations": 1000}},
]

In [ ]:
print(f"\n=== Esecuzione GP su {problems[0]['file_path']} ===")
best_tree = run_gp_on_problem(problems[0]['file_path'], problems[0]['config'])


=== Esecuzione GP su ../data/problem_0.npz ===
Caricamento dati da ../data/problem_0.npz...
Forme originali: X shape (2, 1000), y shape (1000,)
Rilevato formato con features sulle righe e campioni sulle colonne, trasposizione...
X trasposto ha forma (1000, 2)
Forme finali: X shape (1000, 2), y shape (1000,)
Input 2-dimensionale con 1000 campioni

Esecuzione GP su ../data/problem_0.npz...
Avvio Genetic Programming per Symbolic Regression...
Configurazione: pop_size=10000, max_depth=3, generations=1000
Inizializzazione popolazione di 10000 individui...
  Generati 1000 individui in 0.01 secondi
  Generati 2000 individui in 0.03 secondi
  Generati 3000 individui in 0.04 secondi
  Generati 4000 individui in 0.06 secondi
  Generati 5000 individui in 0.08 secondi
  Generati 6000 individui in 0.09 secondi
  Generati 7000 individui in 0.15 secondi
  Generati 8000 individui in 0.16 secondi
  Generati 9000 individui in 0.18 secondi
  Generati 10000 individui in 0.19 secondi
Popolazione inizializ

In [ ]:
print(f"\n=== Esecuzione GP su {problems[1]['file_path']} ===")
best_tree = run_gp_on_problem(problems[1]['file_path'], problems[1]['config'])

In [ ]:
print(f"\n=== Esecuzione GP su {problems[2]['file_path']} ===")
best_tree = run_gp_on_problem(problems[2]['file_path'], problems[2]['config'])

In [ ]:
print(f"\n=== Esecuzione GP su {problems[3]['file_path']} ===")
best_tree = run_gp_on_problem(problems[3]['file_path'], problems[3]['config'])

In [ ]:
print(f"\n=== Esecuzione GP su {problems[4]['file_path']} ===")
best_tree = run_gp_on_problem(problems[4]['file_path'], problems[4]['config'])

In [ ]:
print(f"\n=== Esecuzione GP su {problems[5]['file_path']} ===")
best_tree = run_gp_on_problem(problems[5]['file_path'], problems[5]['config'])

In [ ]:
print(f"\n=== Esecuzione GP su {problems[6]['file_path']} ===")
best_tree = run_gp_on_problem(problems[6]['file_path'], problems[6]['config'])

In [ ]:
print(f"\n=== Esecuzione GP su {problems[7]['file_path']} ===")
best_tree = run_gp_on_problem(problems[7]['file_path'], problems[7]['config'])

In [ ]:
print(f"\n=== Esecuzione GP su {problems[8]['file_path']} ===")
best_tree = run_gp_on_problem(problems[8]['file_path'], problems[8]['config'])    